# Análise de Sentimento simples com a blibioteca TextBlob e Spark Streaming


##  Leitura por meio de socket

###  Seção de Conexão

In [63]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = (
    SparkSession
    .builder
    .appName("analise-sentimento-igti")
    .getOrCreate()
)

## Bibliotecas para a analise e contagem

In [64]:
from textblob import TextBlob
from googletrans import Translator
from unidecode import unidecode

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType


In [65]:
#Dataframe que irá ler cada linha recebida pelo localhost e definição da fonte de dados
twitters = \
    (
        spark
        .readStream
        .format("socket")
        .option("host","localhost")
        .option("port", 9995)
        .load()
    )

## Função para traduzir para o inglês

In [66]:
def translate_udf(col):
    translate_obj = Translator().translate(col)
    return translate_obj.text

## Função para traduzir para realizar a análise de sentimento


In [67]:
def sentiment_udf(col):
    sentiment_text = TextBlob(col)
    return sentiment_text.polarity()

## Difinição das Funções como User-Defined-Function

In [68]:
unicode_udf_string = udf(lambda z: unidecode(z), StringType())
group_by_sentiment = udf(lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutro', StringType()) #classifica o sentimento
translate_udf_string = udf(translate_udf, StringType()) #define função de tradução
sentiment_udf_float = udf(sentiment_udf, FloatType()) #define função de contagem

### teste com texto local

In [69]:
teste = "odeio que não gosta de cachorro."
decode=unidecode(teste)
print(decode)
decodeEN = Translator().translate(decode)
print(decodeEN.text)
a = str(decodeEN)
sentiment = TextBlob(a)
print(sentiment.polarity)

odeio que nao gosta de cachorro.
I hate that you don't like dogs.
-0.8


## Aplicação das funções udf para a seleção de colunas


In [70]:
twitters_unicode = twitters.select( "value", unicode_udf_string(twitters.value). alias("unicoded")) #decodifica
twitters_uni_trans = twitters_unicode.select("value", "unicoded"
                                             , translate_udf_string(col("unicoded")).alias("twitter_EN")) #traduz
twitters_uni_trans_sent = twitters_uni_trans.select("value", "unicoded", "twitter_EN"
                                                    ,sentiment_udf_float(col("twitter_EN")).alias("analise")) #análise de sentimento
t_sent_label = twitters_uni_trans_sent.select("value", "unicoded", "twitter_EN", "analise"
                                              ,group_by_sentiment(col("analise")).alias("classificacao"))

In [71]:
t_sent_count = t_sent_label.groupBy("classificacao").count()

## Definição da querie e como deve ser realizada a saída(sink) para o stram criado

In [72]:
query =t_sent_count \
    .writeStream\
    .outputMode("update")\
    .format("console")\
    .start()

query.awaitTermination()

StreamingQueryException: Writing job aborted.
=== Streaming Query ===
Identifier: [id = ac2f4bd3-66e1-4285-8c98-b393b9376a50, runId = 7a6d2507-8a4a-48ea-9623-e5b27cb9fa3a]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 9995]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=true]
+- Aggregate [classificacao#252], [classificacao#252, count(1) AS count#264L]
   +- Project [value#234, unicoded#237, twitter_EN#241, analise#246, <lambda>(analise#246) AS classificacao#252]
      +- Project [value#234, unicoded#237, twitter_EN#241, sentiment_udf(twitter_EN#241) AS analise#246]
         +- Project [value#234, unicoded#237, translate_udf(unicoded#237) AS twitter_EN#241]
            +- Project [value#234, <lambda>(value#234) AS unicoded#237]
               +- StreamingDataSourceV2Relation [value#234], org.apache.spark.sql.execution.streaming.sources.TextSocketTable$$anon$